# Группировка данных и оконные функции (vo_HW)

## 1. Вывести распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества. — (1 балл)

Сначала проверим на нулевые значения в столбце `job_industry_category`:

In [ ]:
SELECT *
FROM customer c 
WHERE job_industry_category IS NULL;

Нулевых значений нет, тогда выводим распределение (количество) клиентов по сферам деятельности, отсортировав результат по убыванию количества:

In [ ]:
SELECT 
    job_industry_category,
    COUNT(customer_id) as client_count
FROM 
    customer c 
GROUP BY 
    job_industry_category
ORDER BY 
    client_count DESC;

![](imgs/image_01.png)

## 2. Найти сумму транзакций за каждый месяц по сферам деятельности, отсортировав по месяцам и по сфере деятельности. — (1 балл)

Переводим атрибут `transaction_date` из типа *varchar* в тип *date*:

In [ ]:
ALTER TABLE transaction 
ALTER COLUMN transaction_date TYPE DATE 
USING TO_DATE(transaction_date, 'DD.MM.YYYY');

Переводим атрибут `list_price` из типа *varchar* в тип *float*:

In [ ]:
ALTER TABLE transaction
ALTER COLUMN list_price TYPE FLOAT 
USING REPLACE(list_price, ',', '.')::float;

Выводим искомые данные:

In [ ]:
select 
	c.job_industry_category,
	extract (month from t.transaction_date) as tran_month,
	--extract (year from t.transaction_date) as tran_year,
	sum (t.list_price) as tran_sum
from 
	transaction  t
left join 
	customer c on t.customer_id = c.customer_id
group by 
	c.job_industry_category ,
	extract (month from t.transaction_date)
	--extract (year from t.transaction_date)
order by
	tran_month,
	c.job_industry_category

**Примечание:** так как год в данных только 2017, я данную информацию закомментировал.

![](imgs/image_02.png)

## 3. Вывести количество онлайн-заказов для всех брендов в рамках подтвержденных заказов клиентов из сферы IT. — (1 балл)

In [ ]:
select 
	t.brand,
	count (t.transaction_id )
from 
	transaction t 
left join 
	customer c on t.customer_id = c.customer_id
where 
	t.online_order and
	t.order_status = 'Approved' and
	c.job_industry_category = 'IT'
group by 
	brand

![](imgs/image_03.png)

## 4. Найти по всем клиентам сумму всех транзакций (list_price), максимум, минимум и количество транзакций, отсортировав результат по убыванию суммы транзакций и количества клиентов. Выполните двумя способами: используя только group by и используя только оконные функции. Сравните результат. — (2 балла)

Используем `GROUP BY`:

In [ ]:
select 
	t.customer_id ,
	count (t.list_price) as tran_count,
	sum (t.list_price) as tran_sum,
	min (t.list_price) as tran_min,
	max (t.list_price) as tran_max
from 
	transaction t 
group by 	
	t.customer_id
order by 
	tran_sum desc,
	tran_count desc

Используем только оконные функции (в данном случае агрегирующие):

In [ ]:
SELECT distinct
	customer_id, 
	COUNT(list_price) OVER(PARTITION BY customer_id) AS tran_count, 
	SUM(list_price) OVER(PARTITION BY customer_id) AS tran_sum, 
	MIN(list_price) OVER(PARTITION BY customer_id) AS tran_min,
	MAX(list_price) OVER(PARTITION BY customer_id) AS tran_max 	 
FROM transaction
order by 
	tran_sum desc,
	tran_count desc

Результат в обоих случаях идентичен:

![](imgs/image_04.png)

## 5. Найти имена и фамилии клиентов с минимальной/максимальной суммой транзакций за весь период (сумма транзакций не может быть null). Напишите отдельные запросы для минимальной и максимальной суммы.

Клиент с минимальной сумой транзакций:

In [ ]:
select 
	c.customer_id,
	c.first_name,
	c.last_name,
	sum(t.list_price) as sum_amount
from 
	transaction t
left join 
	customer c on t.customer_id = c.customer_id
group by 
	c.customer_id,
	c.first_name,
	c.last_name
ORDER BY 
    sum_amount ASC
LIMIT 1;

![](imgs/image_05_1.png)

Клиент с максимальной сумой транзакций:

In [ ]:
select 
	c.customer_id,
	c.first_name,
	c.last_name,
	sum(t.list_price) as sum_amount
from 
	transaction t
left join 
	customer c on t.customer_id = c.customer_id
group by 
	c.customer_id,
	c.first_name,
	c.last_name
ORDER BY 
    sum_amount DESC
LIMIT 1;

![](imgs/image_05_2.png)

## 6. Вывести только самые первые транзакции клиентов. Решить с помощью оконных функций. — (1 балл)

In [ ]:
WITH ranked_transactions AS (
    SELECT 
        t.customer_id,
        c.first_name,
        c.last_name,
        t.transaction_date,
        ROW_NUMBER() OVER (PARTITION BY t.customer_id ORDER BY t.transaction_date) AS row_number
    FROM 
        transaction t 
    LEFT JOIN 
        customer c ON t.customer_id = c.customer_id
)
SELECT 
    customer_id,
    first_name,
    last_name,
    transaction_date,
    row_number
FROM 
    ranked_transactions
WHERE 
    row_number = 1;

![](imgs/image_06.png)

## 7. Вывести имена, фамилии и профессии клиентов, между транзакциями которых был максимальный интервал (интервал вычисляется в днях) — (2 балла).

In [ ]:
WITH 
transaction_intervals AS (
    SELECT 
        t.customer_id,
        c.first_name,
        c.last_name,
        c.job_title,
        t.transaction_date,
        LAG(t.transaction_date) OVER (PARTITION BY t.customer_id ORDER BY t.transaction_date) AS prev_transaction_date,
        (t.transaction_date - LAG(t.transaction_date) OVER (PARTITION BY t.customer_id ORDER BY t.transaction_date)) AS interval_days
    FROM 
        transaction t
    JOIN 
        customer c ON t.customer_id = c.customer_id
),

max_intervals AS (
    SELECT 
        customer_id,
        first_name,
        last_name,
        job_title,
        MAX(interval_days) AS max_interval
    FROM 
        transaction_intervals
    WHERE 
        interval_days IS NOT NULL
    GROUP BY 
        customer_id,
        first_name,
        last_name,
        job_title
)

SELECT 
	customer_id,
    first_name,
    last_name,
    job_title,
    max_interval
FROM 
    max_intervals
WHERE 
    max_interval = (SELECT MAX(max_interval) FROM max_intervals)


![](imgs/image_07.png)